In [1]:
# LSTM Bidirectional RNN
import pandas as pd

In [3]:
data = pd.read_csv('Data/train.csv')
# 1 -> Fake
# 0 -> Not Fake

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
##Drop Nan Values
data = data.dropna()

In [6]:
# Remove label from data

X = data.drop('label', axis=1)

In [7]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
# Store results

Y = data['label']

In [9]:
X.shape

(18285, 4)

In [10]:
Y.shape

(18285,)

In [11]:
Y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
# Vocabulary size
vocab_size = 5000

In [14]:
# Onehot Representation
messages = X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace = True)

In [17]:
# Stemming and stop word removal

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
#     type(messages['title'][i])
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [19]:
# One Hot encoding 

onehot_data = [one_hot(words, vocab_size) for words in corpus] 
onehot_data

[[4773, 4348, 3994, 1413, 468, 3437, 4574, 1616, 4427, 4890],
 [1137, 986, 1986, 3076, 252, 4713, 3915],
 [1333, 2939, 4877, 753],
 [2042, 1896, 2219, 211, 3666, 1536],
 [1948, 252, 4763, 2098, 3970, 3757, 252, 3624, 2479, 1115],
 [57,
  2629,
  2498,
  2983,
  2465,
  4347,
  1868,
  1375,
  1255,
  3030,
  1801,
  4386,
  244,
  1857,
  3915],
 [2701, 72, 4769, 4535, 1826, 3599, 2079, 2707, 1114, 2316, 2642],
 [3259, 1783, 2108, 1431, 930, 4828, 4347, 2395, 1114, 2316, 2642],
 [3071, 4326, 2929, 4861, 4511, 328, 1775, 4356, 4347, 1347],
 [2292, 2654, 1680, 3663, 263, 3121, 2820, 1708],
 [2284, 420, 2274, 4457, 2502, 4178, 4263, 1452, 2149, 624, 3136],
 [211, 4191, 468, 328, 4347, 930],
 [1229, 2226, 632, 3570, 3057, 708, 4204, 3090, 2014],
 [2252, 2460, 1267, 1360, 2523, 3386, 3217, 1114, 2316, 2642],
 [2096, 1867, 924, 1550, 1667, 1114, 2316, 2642],
 [523, 220, 196, 3183, 3464, 4580, 2334, 3598, 4579, 4821],
 [934, 592, 986],
 [3151, 2708, 3995, 2035, 4347, 3997, 461, 3915],
 [3658,

In [20]:
# Embedding

sentence_length = 20
embedded_data = pad_sequences(onehot_data, padding='pre', maxlen = sentence_length)
print(embedded_data)

[[   0    0    0 ... 1616 4427 4890]
 [   0    0    0 ...  252 4713 3915]
 [   0    0    0 ... 2939 4877  753]
 ...
 [   0    0    0 ... 1114 2316 2642]
 [   0    0    0 ... 1471 2885 3397]
 [   0    0    0 ... 4401 2784 2580]]


In [21]:
embedded_data[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4773,
       4348, 3994, 1413,  468, 3437, 4574, 1616, 4427, 4890])

In [24]:
# Creating Bidirectional LSTM model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sentence_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
# Creating second model

embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_vector_features, input_length = sentence_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model1.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
len(embedded_data), Y.shape

(18285, (18285,))

In [30]:
import numpy as np

X_final = np.array(embedded_data)
Y_final = np.array(Y)

In [31]:
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, Y_final, test_size = 0.33, random_state = 42)

In [33]:
len(X_train), len(y_train), len(X_test), len(y_test)

(12250, 12250, 6035, 6035)

In [34]:
# Training

model1.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 64, epochs = 10)

Epoch 1/10
192/192 [==============================] - 6s 30ms/step - loss: 0.2991 - accuracy: 0.8596 - val_loss: 0.2025 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1362 - accuracy: 0.9473 - val_loss: 0.1978 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0943 - accuracy: 0.9665 - val_loss: 0.2553 - val_accuracy: 0.9143
Epoch 4/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0708 - accuracy: 0.9761 - val_loss: 0.2523 - val_accuracy: 0.9114
Epoch 5/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0559 - accuracy: 0.9819 - val_loss: 0.3565 - val_accuracy: 0.9130
Epoch 6/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0386 - accuracy: 0.9873 - val_loss: 0.3816 - val_accuracy: 0.9082
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0360 - accuracy: 0.9882 - val_loss: 0.3655 - val_accuracy: 0.9059

In [36]:
# Prediction

y_pred = model1.predict(X_test)
y_pred

array([[9.9983108e-01],
       [1.6955435e-03],
       [4.2164326e-04],
       ...,
       [1.0189833e-03],
       [9.9904937e-01],
       [9.9989760e-01]], dtype=float32)

In [37]:
# Performance and Accuracy
predictions = []
for i in range(len(y_pred)):
    if y_pred[i].item() > 0.5:
#         predictions.append(str(i) + " is Fake " + str(1))
        predictions.append(1)
    else:
#         predictions.append(str(i) +" is Not Fake " + str(0))
        predictions.append(0)

In [38]:
predictions

[1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,


In [39]:
results = model1.evaluate(X_test, y_test, batch_size = 64)
print("test loss, test acc:", results)

95/95 [==============================] - 1s 9ms/step - loss: 0.5185 - accuracy: 0.9082
test loss, test acc: [0.5185409188270569, 0.9082021713256836]


In [40]:
# Creating classification report 
from sklearn import metrics

print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

